In [2]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import sys
sys.path.append("..")
from utils import config

In [3]:
def encode(A):
	classes = np.unique(A)
	dictionary = dict(zip(classes, range(len(classes))))
	A_numeric = list(map(dictionary.get, A))
	return(A_numeric, dictionary)

In [4]:
con = sqlite3.connect(config.DATABASE_PATH) 

query = '''
SELECT c.CountryCode, i.IndicatorCode,ci.Year, ci.Value
FROM CountryIndicators as ci, Countries as c, Indicators as i
WHERE ci.CountryCode=c.CountryCode AND i.IndicatorCode=ci.IndicatorCode  ;
'''

table = pd.read_sql_query(query, con)

con.close()

In [44]:
print(table.shape)
table.head()

(5026221, 4)


,CountryCode,IndicatorCode,Year,Value
0,ARB,SP.ADO.TFRT,1960,1.335609e+02
1,ARB,SP.POP.DPND,1960,8.779760e+01
2,ARB,SP.POP.DPND.OL,1960,6.634579e+00
3,ARB,SP.POP.DPND.YG,1960,8.102333e+01
4,ARB,MS.MIL.XPRT.KD,1960,3.000000e+06


In [6]:
country_codes = pd.DataFrame(set(table['CountryCode']))

In [45]:
country_codes

,0
0,SRB
1,AGO
2,ALB
3,KIR
4,IRL
...,...
242,SYR
243,VNM
244,SLE
245,DOM


In [7]:
country_indicators_table = table.pivot_table(index = ['CountryCode','Year'], columns = "IndicatorCode", values = 'Value')
country_indicators_table['nextyearGDPgrowth'] = country_indicators_table['NY.GDP.MKTP.KD.ZG'].shift(periods = -1)
#test.to_sql("formatted", con, if_exists="replace")

In [46]:
country_indicators_table

IndicatorCode     SI.SPR.PCAP.ZG  SI.SPR.PC40.ZG  SI.SPR.PC40.05  \
CountryCode Year                                                   
ABW         1960             NaN             NaN             NaN   
            1961             NaN             NaN             NaN   
            1962             NaN             NaN             NaN   
            1963             NaN             NaN             NaN   
            1964             NaN             NaN             NaN   
...                          ...             ...             ...   
ZWE         2006             NaN             NaN             NaN   
            2007             NaN             NaN             NaN   
            2008             NaN             NaN             NaN   
            2009             NaN             NaN             NaN   
            2010             NaN             NaN             NaN   

IndicatorCode     SI.SPR.PCAP.05  LP.LPI.LOGS.XQ  LP.LPI.TRAC.XQ  \
CountryCode Year                                                   
ABW         1960             NaN             NaN             NaN   
            1961             NaN             NaN             NaN   
            1962             NaN             NaN             NaN   
            1963             NaN             NaN             NaN   
            1964             NaN             NaN             NaN   
...                          ...             ...             ...   
ZWE         2006             NaN             NaN             NaN   
            2007             NaN            2.21            2.64   
            2008             NaN             NaN             NaN   
            2009             NaN             NaN             NaN   
            2010             NaN             NaN             NaN   

IndicatorCode     LP.LPI.CUST.XQ  LP.LPI.INFR.XQ  LP.LPI.OVRL.XQ  \
CountryCode Year                                                   
ABW         1960             NaN             NaN             NaN   
            1961             NaN             NaN             NaN   
            1962             NaN             NaN             NaN   
            1963             NaN             NaN             NaN   
            1964             NaN             NaN             NaN   
...                          ...             ...             ...   
ZWE         2006             NaN             NaN             NaN   
            2007            1.92            1.87            2.29   
            2008             NaN             NaN             NaN   
            2009             NaN             NaN             NaN   
            2010             NaN             NaN             NaN   

IndicatorCode     per_allsp.ben_q1_tot  ...  NV.IND.TOTL.KD.ZG  IC.EXP.DURS  \
CountryCode Year                        ...                                   
ABW         1960                   NaN  ...                NaN          NaN   
            1961                   NaN  ...                NaN          NaN   
            1962                   NaN  ...                NaN          NaN   
            1963                   NaN  ...                NaN          NaN   
            1964                   NaN  ...                NaN          NaN   
...                                ...  ...                ...          ...   
ZWE         2006                   NaN  ...          -4.152000         52.0   
            2007                   NaN  ...          -3.767697         52.0   
            2008                   NaN  ...         -20.914062         53.0   
            2009                   NaN  ...           9.660698         53.0   
            2010                   NaN  ...          14.887979         53.0   

IndicatorCode     IC.TAX.GIFT.ZS  IQ.CPA.FISP.XQ  IQ.WEF.PORT.XQ  \
CountryCode Year                                                   
ABW         1960             NaN             NaN             NaN   
            1961             NaN             NaN             NaN   
            1962             NaN             

In [8]:
print(country_indicators_table.shape)
country_indicators_table.tail()

(12597, 1329)


IndicatorCode     AG.AGR.TRAC.NO  AG.CON.FERT.PT.ZS  AG.CON.FERT.ZS  \
CountryCode Year                                                      
ZWE         2006             NaN         210.442742       32.356341   
            2007             NaN         171.109177       27.035250   
            2008             NaN         147.903481       21.994118   
            2009             NaN         186.693038       28.778049   
            2010             NaN         215.715190       34.083000   

IndicatorCode     AG.LND.AGRI.K2  AG.LND.AGRI.ZS  AG.LND.ARBL.HA  \
CountryCode Year                                                   
ZWE         2006        163000.0       42.135195       4100000.0   
            2007        162000.0       41.876696       4000000.0   
            2008        164500.0       42.522942       4250000.0   
            2009        163000.0       42.135195       4100000.0   
            2010        162000.0       41.876696       4000000.0   

IndicatorCode     AG.LND.ARBL.HA.PC  AG.LND.ARBL.ZS  AG.LND.CREL.HA  \
CountryCode Year                                                      
ZWE         2006           0.312311       10.598423       2289312.0   
            2007           0.300802       10.339925       1949453.0   
            2008           0.314921       10.986170       2235440.0   
            2009           0.298812       10.598423       2080405.0   
            2010           0.286248       10.339925       1891766.0   

IndicatorCode     AG.LND.CROP.ZS  ...  per_lm_alllm.adq_pop_tot  \
CountryCode Year                  ...                             
ZWE         2006        0.258498  ...                       NaN   
            2007        0.258498  ...                       NaN   
            2008        0.258498  ...                       NaN   
            2009        0.258498  ...                       NaN   
            2010        0.258498  ...                       NaN   

IndicatorCode     per_lm_alllm.ben_q1_tot  per_lm_alllm.cov_pop_tot  \
CountryCode Year                                                      
ZWE         2006                      NaN                       NaN   
            2007                      NaN                       NaN   
            2008                      NaN                       NaN   
            2009                      NaN                       NaN   
            2010                      NaN                       NaN   

IndicatorCode     per_sa_allsa.adq_pop_tot  per_sa_allsa.ben_q1_tot  \
CountryCode Year                                                      
ZWE         2006                       NaN                      NaN   
            2007                       NaN                      NaN   
            2008                       NaN                      NaN   
            2009                       NaN                      NaN   
            2010                       NaN                      NaN   

IndicatorCode     per_sa_allsa.cov_pop_tot  per_si_allsi.adq_pop_tot  \
CountryCode Year                                                       
ZWE         2006                       NaN                       NaN   
            2007                       NaN                       NaN   
            2008                       NaN                       NaN   
            2009                       NaN                       NaN   
            2010                       NaN                       NaN   

IndicatorCode     per_si_allsi.ben_q1_tot  per_si_allsi.cov_pop_tot  \
CountryCode Year                                                      
ZWE         2006                      NaN                       NaN   
            2007                      NaN                       NaN   
            2008                      NaN                       NaN   
            2009                      NaN                       NaN   
            2010                      NaN                       NaN   

IndicatorCode     nextyearGDPgrowth  
CountryCode Year    

As we can see, 

In [9]:
cor_matrix_indicators = country_indicators_table.corr()

In [10]:
cor_vector_indicators = cor_matrix_indicators["nextyearGDPgrowth"]

In [11]:
df_indicators_final = pd.DataFrame (cor_vector_indicators, index = cor_matrix_indicators.columns)
df_indicators_final['nextyearGDPgrowth.abs'] = abs(cor_vector_indicators)
df_indicators_final = df_indicators_final.sort_values(by='nextyearGDPgrowth.abs',ascending=0).head(50)

In [12]:
indicadors = df_indicators_final.index

In [13]:
country_indicators_table=country_indicators_table[indicadors]
country_indicators_table.tail()

IndicatorCode     nextyearGDPgrowth  SI.SPR.PCAP.ZG  SI.SPR.PC40.ZG  \
CountryCode Year                                                      
ZWE         2006          -3.653327             NaN             NaN   
            2007         -17.668947             NaN             NaN   
            2008           5.984391             NaN             NaN   
            2009          11.375921             NaN             NaN   
            2010                NaN             NaN             NaN   

IndicatorCode     SI.SPR.PC40.05  SI.SPR.PCAP.05  LP.LPI.LOGS.XQ  \
CountryCode Year                                                   
ZWE         2006             NaN             NaN             NaN   
            2007             NaN             NaN            2.21   
            2008             NaN             NaN             NaN   
            2009             NaN             NaN             NaN   
            2010             NaN             NaN             NaN   

IndicatorCode     LP.LPI.TRAC.XQ  LP.LPI.CUST.XQ  LP.LPI.INFR.XQ  \
CountryCode Year                                                   
ZWE         2006             NaN             NaN             NaN   
            2007            2.64            1.92            1.87   
            2008             NaN             NaN             NaN   
            2009             NaN             NaN             NaN   
            2010             NaN             NaN             NaN   

IndicatorCode     LP.LPI.OVRL.XQ  ...  IC.IMP.DURS  NV.IND.TOTL.KD.ZG  \
CountryCode Year                  ...                                   
ZWE         2006             NaN  ...         67.0          -4.152000   
            2007            2.29  ...         67.0          -3.767697   
            2008             NaN  ...         73.0         -20.914062   
            2009             NaN  ...         73.0           9.660698   
            2010             NaN  ...         73.0          14.887979   

IndicatorCode     IC.EXP.DURS  IC.TAX.GIFT.ZS  IQ.CPA.FISP.XQ  IQ.WEF.PORT.XQ  \
CountryCode Year                                                                
ZWE         2006         52.0             NaN             1.0             NaN   
            2007         52.0             NaN             1.0        4.083333   
            2008         53.0             NaN             1.0        4.270329   
            2009         53.0             NaN             2.0        4.417178   
            2010         53.0             NaN             2.0        4.409526   

IndicatorCode     NE.IMP.GNFS.KD.ZG  FS.AST.DOMS.GD.ZS  NE.CON.PRVT.KD.ZG  \
CountryCode Year                                                            
ZWE         2006          -6.680595                NaN           4.992830   
            2007         -12.666982                NaN          -6.923020   
            2008          11.647857                NaN          -3.137561   
            2009          40.964584                NaN          31.877126   
            2010          53.386946                NaN          -0.096450   

IndicatorCode     SH.MED.BEDS.ZS  
CountryCode Year                  
ZWE         2006             3.0  
            2007             NaN  
            2008             NaN  
            2009             NaN  
            2010             NaN  

[5 rows x 50 columns]

In [14]:
nextyearGDP = country_indicators_table.pop("nextyearGDPgrowth")
country_indicators_table["nextyearGDPgrowth"]=nextyearGDP

In [15]:
country_indicators_table.shape

(12597, 50)

In [16]:
classList = np.array(list(map(list, country_indicators_table.index)))

'''
country_indicators_table[:,0] #numeric country code
country_indicators_table[:,1] #year
country_indicators_table[:,-1] #GDP to predict
'''

'\ncountry_indicators_table[:,0] #numeric country code\ncountry_indicators_table[:,1] #year\ncountry_indicators_table[:,-1] #GDP to predict\n'

In [17]:
countryCodes_num, countryDict = encode(classList[:,0])
years_num = np.array(list(map(int, classList[:,1])))[:,np.newaxis]
countryCodes_num = np.array(countryCodes_num)[:,np.newaxis]

In [47]:
countryCodes_num

array([[  0],
       [  0],
       [  0],
       ...,
       [246],
       [246],
       [246]])

In [48]:
countryDict

{'ABW': 0,
 'ADO': 1,
 'AFG': 2,
 'AGO': 3,
 'ALB': 4,
 'ARB': 5,
 'ARE': 6,
 'ARG': 7,
 'ARM': 8,
 'ASM': 9,
 'ATG': 10,
 'AUS': 11,
 'AUT': 12,
 'AZE': 13,
 'BDI': 14,
 'BEL': 15,
 'BEN': 16,
 'BFA': 17,
 'BGD': 18,
 'BGR': 19,
 'BHR': 20,
 'BHS': 21,
 'BIH': 22,
 'BLR': 23,
 'BLZ': 24,
 'BMU': 25,
 'BOL': 26,
 'BRA': 27,
 'BRB': 28,
 'BRN': 29,
 'BTN': 30,
 'BWA': 31,
 'CAF': 32,
 'CAN': 33,
 'CEB': 34,
 'CHE': 35,
 'CHI': 36,
 'CHL': 37,
 'CHN': 38,
 'CIV': 39,
 'CMR': 40,
 'COG': 41,
 'COL': 42,
 'COM': 43,
 'CPV': 44,
 'CRI': 45,
 'CSS': 46,
 'CUB': 47,
 'CUW': 48,
 'CYM': 49,
 'CYP': 50,
 'CZE': 51,
 'DEU': 52,
 'DJI': 53,
 'DMA': 54,
 'DNK': 55,
 'DOM': 56,
 'DZA': 57,
 'EAP': 58,
 'EAS': 59,
 'ECA': 60,
 'ECS': 61,
 'ECU': 62,
 'EGY': 63,
 'EMU': 64,
 'ERI': 65,
 'ESP': 66,
 'EST': 67,
 'ETH': 68,
 'EUU': 69,
 'FCS': 70,
 'FIN': 71,
 'FJI': 72,
 'FRA': 73,
 'FRO': 74,
 'FSM': 75,
 'GAB': 76,
 'GBR': 77,
 'GEO': 78,
 'GHA': 79,
 'GIN': 80,
 'GMB': 81,
 'GNB': 82,
 'GNQ': 83,
 '

In [49]:
years_num

array([[1960],
       [1961],
       [1962],
       ...,
       [2008],
       [2009],
       [2010]])

In [19]:
values = country_indicators_table.to_numpy()
country_indicators= np.concatenate((countryCodes_num,years_num,values), axis=1)

In [50]:
values

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ..., -3.13756141,
                nan,  5.98439075],
       [        nan,         nan,         nan, ..., 31.87712627,
                nan, 11.37592123],
       [        nan,         nan,         nan, ..., -0.09645039,
                nan,         nan]])

In [20]:
country_indicators

array([[ 0.00000000e+00,  1.96000000e+03,             nan, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  1.96100000e+03,             nan, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  1.96200000e+03,             nan, ...,
                    nan,             nan,             nan],
       ...,
       [ 2.46000000e+02,  2.00800000e+03,             nan, ...,
        -3.13756141e+00,             nan,  5.98439075e+00],
       [ 2.46000000e+02,  2.00900000e+03,             nan, ...,
         3.18771263e+01,             nan,  1.13759212e+01],
       [ 2.46000000e+02,  2.01000000e+03,             nan, ...,
        -9.64503944e-02,             nan,             nan]])

QUITAMOS LOS NANs #Reemplazamos los Nan por la media correspondiente de cada columna

In [21]:
country_indicators_mean = np.nanmean(country_indicators, axis=0)
inds = np.where(np.isnan(country_indicators))
country_indicators[inds] = np.take(country_indicators_mean, inds[1])
country_indicators_mean = country_indicators

In [22]:

indexes_2010_mean = np.where((country_indicators_mean[:,1]==2010))
country_indicators_2010_mean = country_indicators_mean[indexes_2010_mean]

country_indicators_no2010_mean = np.delete(country_indicators_mean, indexes_2010_mean, 0)

In [23]:
X_2010_mean = country_indicators_2010_mean[:,:-1]

X_mean = country_indicators_no2010_mean[:,:-1]
y_mean = country_indicators_no2010_mean[:,-1]

In [24]:
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X_mean,y_mean,test_size=0.2)
print(X_train_mean.shape)
print(X_test_mean.shape)
print(y_train_mean.shape)
print(y_test_mean.shape)

(9880, 51)
(2470, 51)
(9880,)
(2470,)


In [25]:
gbreg_es_mean = GradientBoostingRegressor(max_depth=2, n_estimators=2000,
                                     validation_fraction=0.05,n_iter_no_change=25,
                                     learning_rate=0.1, random_state=42,criterion="mse")

In [26]:
gbreg_es_mean = GradientBoostingRegressor(max_depth=10, n_estimators=100,
                                     learning_rate=0.1,loss="ls")

In [27]:
gbreg_es_mean.fit(X_train_mean,y_train_mean)
print('n_estimators_:',gbreg_es_mean.n_estimators_)
print('MSE is:',np.sqrt(mean_squared_error(y_test_mean,gbreg_es_mean.predict(X_test_mean))))

n_estimators_: 100
MSE is: 5.3679141859730715


In [28]:
pred_vs_real_mean = pd.DataFrame(y_test_mean,columns=['real'])
pred_vs_real_mean['predictions'] = gbreg_es_mean.predict(X_test_mean)

In [29]:
pred_vs_real_mean.head()

,real,predictions
0,1.269758,2.554372
1,6.511970,5.153714
2,-1.251597,2.885388
3,11.018512,2.711637
4,3.974739,3.460978


In [30]:
gbreg_es_mean.predict(X_2010_mean)

array([ 3.97517717e+00,  1.24919527e+00,  7.00718902e+00,  3.76242616e+00,
        3.36767870e+00,  4.50707021e+00,  6.55737919e+00,  6.83345492e+00,
        3.33736690e+00,  3.97517717e+00, -3.13129187e+00,  3.60962221e+00,
        3.12333593e+00,  5.97809925e+00,  7.48602656e+00,  1.55839894e+00,
        4.47817216e+00,  6.31272688e+00,  5.02167494e+00,  4.70929551e-01,
        2.44381645e+00,  2.76420251e+00,  2.52637083e+00,  6.24072276e+00,
        1.85441422e+00,  6.15323592e-01,  3.53679454e+00,  5.99437273e+00,
        2.20555806e+00,  1.15336951e+00,  9.90732727e+00,  7.87600011e+00,
        5.92579859e+00,  1.88285501e+00,  2.49812907e+00,  1.44478117e+00,
        3.93074453e+00,  5.89016369e+00,  1.00312363e+01,  3.72633473e+00,
        3.69836587e+00,  4.97930682e+00,  3.63400143e+00,  4.76922740e+00,
        3.11696452e+00,  2.14130833e+00,  3.51078953e+00,  3.12357445e+00,
        3.93074453e+00,  1.21065952e-02,  4.25072966e+00,  2.47413021e+00,
        2.69898802e+00,  

QUITAMOS LOS NANs #Reemplazamos los Nan por 0

In [31]:
country_indicators_0 = np.nan_to_num(country_indicators, nan = 0.0)

In [32]:
indexes_2010_0 = np.where((country_indicators_0[:,1]==2010))
country_indicators_2010_0 = country_indicators_0[indexes_2010_0]

country_indicators_no2010_0 = np.delete(country_indicators_0, indexes_2010_0, 0)

In [33]:
X_2010_0 = country_indicators_2010_0[:,:-1]

X_0 = country_indicators_no2010_0[:,:-1]
y_0 = country_indicators_no2010_0[:,-1]

In [34]:
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0,y_0,test_size=0.2)
print(X_train_0.shape)
print(X_test_0.shape)
print(y_train_0.shape)
print(y_test_0.shape)

(9880, 51)
(2470, 51)
(9880,)
(2470,)


In [35]:
gbreg_es_0 = GradientBoostingRegressor(max_depth=2, n_estimators=2000,
                                     validation_fraction=0.05,n_iter_no_change=25,
                                     learning_rate=0.1, random_state=42,criterion="mse")

In [36]:
gbreg_es_0.fit(X_train_0,y_train_0)
print('n_estimators_:',gbreg_es_0.n_estimators_)
print('MSE is:',np.sqrt(mean_squared_error(y_test_0,gbreg_es_0.predict(X_test_0))))

n_estimators_: 116
MSE is: 5.323071780250422


In [37]:
pred_vs_real_0 = pd.DataFrame(y_test_0,columns=['real'])
pred_vs_real_0['predictions'] = gbreg_es_0.predict(X_test_0)

In [38]:
pred_vs_real_0.head()

,real,predictions
0,12.064409,4.402542
1,3.436426,3.437717
2,3.396535,3.681704
3,0.955757,-0.608361
4,4.468129,3.392927


In [39]:
gbreg_es_0.predict(X_2010_0)

array([ 4.2272525 ,  2.57661396,  6.84834431,  4.85887628,  3.98184919,
        4.50396615,  2.96267434,  6.94266137,  4.04357193,  4.2272525 ,
       -0.14847554,  3.10139352,  3.28897763,  4.71806458,  5.68310865,
        3.64129818,  3.7322873 ,  7.15801802,  5.11048702,  1.9244177 ,
        4.69982771,  3.47778493,  3.05589523,  7.64292166,  3.89665882,
        1.61331423,  4.83214284,  6.20523953,  3.269934  ,  3.2334614 ,
       15.49791333,  6.14394679,  4.59529575,  3.81178476,  3.33981674,
        3.62420065,  4.2272525 ,  5.0660983 ,  8.07734861,  3.38846165,
        4.85225169,  7.06723225,  4.60145238,  4.92696166,  3.31675186,
        4.85494053,  2.99316081,  4.20210532,  4.2272525 ,  3.26395277,
        2.72540874,  3.75658183,  4.41661809,  4.20425531,  3.37893635,
        2.77941581,  6.5907213 ,  4.27711673,  6.7560456 ,  6.17632765,
        5.5449466 ,  3.33981674,  4.53212509,  5.2782856 ,  3.28897763,
        4.34733678,  1.9244177 ,  4.00915898,  8.86280941,  3.33

NO HACE FALTA QUITAR NANs, usamos otra libreria LightGBM

In [40]:
indexes_2010 = np.where((country_indicators[:,1]==2010))
country_indicators_2010 = country_indicators[indexes_2010]

country_indicators_no2010 = np.delete(country_indicators, indexes_2010, 0)

In [41]:
X_2010 = country_indicators_2010[:,:-1]

X = country_indicators_no2010[:,:-1]
y = country_indicators_no2010[:,-1]

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9880, 51)
(2470, 51)
(9880,)
(2470,)


In [43]:
import lightgbm as lgb

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
gbreg_es = lgb.LGBMRegressor(boosting_type = 'gbdt', objective = 'regression', num_leaves = 1200,
                                learning_rate = 0.1, n_estimators = 2000, max_depth = 2,
                                metric = 'mse', bagging_fraction = 0.8, feature_fraction = 0.8, reg_lambda = 0.9)

gbreg_es.fit(X_train, y_train)

In [ ]:
print('MSE is:',np.sqrt(mean_squared_error(y_test,gbreg_es.predict(X_test))))

In [117]:
model = lgb.LGBMRegressor()

In [ ]:
model.fit(X_train, y_train)